In [ ]:
class Registry(dict):
    def __init__(self, *args, **kwargs):
        super(Registry,self).__init__(*agrs, **kwargs)

    def register(self, module_name, module=None):
        if module is not None:
            _register_generic(self, module_name, module)
            return
        def register_fn(fn):
            _register_generic(self, module_name, fn)
            return fn
        return register_fn
    
def register_generic(module_dict, module_name, module):
    assert module_name not in module_dict
    module_dict[module_name] = module 
    

In [ ]:
# registry class extends the dictionary data structure and provide and 
# extra register function. This register function is used as a decorator to register
# our module. 
HEAD_REGISTRY = Registry()


In [ ]:
import argparse
parser = argparse.ArgumentParser(description = "Description of your program")
parser.add_argument('directory', help="path to the input file")
args = parser.parse_args()
file_path = args.directory

In [ ]:
def stickler_thief(arr: list):
    sub_array = []
    check_indices = []

    while arr:
        max_element = max(arr)
        max_indices = [ind for ind, ele in enumerate(arr) if ele = max_element]
        if len(max_indices) > 0 :
            max_indices = max_indices.sort()
            if any([(max_indices[i]-max_indices[i+1])==1 for i in range(len(max_indices))]):
                check_indices.extend([[indices-1, indices+1] for indices in max_indices])
                check_val = [arr[i] for i in check_indices]
                check_val = min(check_val)
                min_ind = 








               
            

            




In [ ]:
# Tutorial on Pytorch gather
# torch.gather(input, dim, index, *, sparse_grad=False, out=None) → Tensor
# --> dim = the dimension along which gather occurs
# --> index = tensor specifying the gathering rules

import torch
M = torch.tensor([[1,2,3], [4,7,9], [19,9,23]]) 
indexes = torch.tensor([1,1,2])
indexes = indexes.unsqueeze(0)
print(indexes.shape)
out = M.gather(dim=0,index=indexes)

### Understanding pytorch scatter_()

In [ ]:
import torch


# dummy variable for checking
x = torch.rand(64,3,32,32)
mal_num = 6
fixed = torch.zeros_like(x.clone()[:-mal_num], requires_grad=False)
adv = (torch.zeros_like(x.clone()[-mal_num:])- x[-mal_num:] + 127.5/255).requires_grad_(True)
adv_pad = torch.cat((fixed,adv), 0)

In [ ]:
import torch
# dummy variables 
d_batch = torch.rand(64,3,32,32)
d_idx = torch.tensor([11,17,6,5,31,51])
adv_pad = torch.zeros_like(x.clone(),requires_grad=False)

def denorm(self, data_batch, mu, std):
        """
        Convert a batch of tensors to their original scale.
        Args:
            batch (torch.Tensor): Batch of normalized tensors.
            mean (torch.Tensor or list): Mean used for normalization.
            std (torch.Tensor or list): Standard deviation used for normalization.
        Returns:
            torch.Tensor: batch of tensors without normalization applied to them.
        """
        if isinstance(mean, list):
            mean = torch.tensor(mu).to(self.device)
        if isinstance(std, list):
            std = torch.tensor(std).to(self.device)
        return torch.clamp(data_batch * std.view(1, -1, 1, 1) + mean.view(1, -1, 1, 1), 0, 1)


In [ ]:
# Solution 
import torch
B, H, W = 2, 5, 2
x = torch.arange(B*H*W).view(B,H,W)
print(x)
idx = torch.randperm(x.size(1))
print(idx)
y = x[:,idx]
print(y)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(0)

# Here's a simple CNN and loss function:

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

def loss_fn(predictions, targets):
    return F.nll_loss(predictions, targets)

device = 'cuda'

num_models = 10
batch_size = 64
data = torch.randn(batch_size, 1, 28, 28, device=device)

targets = torch.randint(10, (64,), device=device)

model = SimpleCNN().to(device=device)
predictions = model(data)  # move the entire mini-batch through the model

loss = loss_fn(predictions, targets)
loss.backward()  # back propagate the 'average' gradient of this mini-batch

def compute_grad(sample, target):
    sample = sample.unsqueeze(0)  # prepend batch dimension for processing
    target = target.unsqueeze(0)

    prediction = model(sample)
    loss = loss_fn(prediction, target)

    return torch.autograd.grad(loss, list(model.parameters()))


def compute_sample_grads(data, targets):
    """ manually process each sample with per sample gradient """
    sample_grads = [compute_grad(data[i], targets[i]) for i in range(batch_size)]
    sample_grads = zip(*sample_grads)
    sample_grads = [torch.stack(shards) for shards in sample_grads]
    return sample_grads

per_sample_grads = compute_sample_grads(data, targets)
print(per_sample_grads[0].shape)

In [ ]:
from torch.func import functional_call, vmap, grad

params = {k: v.detach() for k, v in model.named_parameters()}
buffers = {k: v.detach() for k, v in model.named_buffers()}

# params = {k: v.detach() for k, v in model.conv2.named_parameters()}
# buffers = {k: v.detach() for k, v in model.conv2.named_buffers()}

def compute_loss(params, buffers, sample, target):
    batch = sample.unsqueeze(0)
    targets = target.unsqueeze(0)

    predictions = functional_call(model, (params, buffers), (batch,))
    loss = loss_fn(predictions, targets)
    return loss
ft_compute_grad = grad(compute_loss)
ft_compute_sample_grad = vmap(ft_compute_grad, in_dims=(None, None, 0, 0))
ft_per_sample_grads = ft_compute_sample_grad(params, buffers, data, targets)
print(ft_per_sample_grads['conv2.weight'].size())

In [35]:
import math
b=128
portion=0.2
mal_num = math.ceil(b*portion)
print(type(mal_num))


<class 'int'>
